## 🚀 미션: “뉴스 토픽 분류 & 언어 모델 실습”

### 배경 스토리  
여러분은 미디어 분석팀의 신입 NLP AI SW 엔지니어입니다.  
매일 쏟아지는 뉴스 기사를 **정치**, **경제**, **스포츠**, **기술**, 기타 등 5가지 카테고리로 자동 분류하고, 동시에 뉴스 기사의 언어모델을 연구하기 위해 “다음 단어”를 예측하는 **언어 모델**을 구축해야 합니다.  

---

### 문제 1. 데이터 준비 및 전처리  
1. **데이터셋**:
BBC articles fulltext and category
https://www.kaggle.com/datasets/yufengdev/bbc-fulltext-and-category?resource=download

2. **Tokenization**: `Tokenizer`를 이용해 시퀀스로 변환하고, `pad_sequences`로 길이 200으로 패딩.  
3. **훈련/검증 분할**: 훈련 데이터를 80%/20%로 나누기.  

> **출력**:  
> - 단어 집합 크기(`num_words`)  
> - 패딩 후 시퀀스 형태  
> - 훈련/검증 샘플 개수  

---

### 문제 2. 랜덤 초기화 임베딩을 이용한 텍스트 분류  
1. **모델 구조**:  
   - `Embedding(input_dim=num_words, output_dim=100, input_length=200)`  
   - `GlobalAveragePooling1D()`  
   - `Dense(64, activation='relu')`  
   - `Dense(4, activation='softmax')`  
2. **학습 설정**:  
   - 옵티마이저: Adam  
   - 손실함수: sparse categorical crossentropy  
   - 평가 지표: accuracy  
   - 에포크: 10, 배치사이즈: 32  
3. **결과 기록**:  
   - 훈련/검증 정확도 및 손실 플롯  

> **질문**: 랜덤 초기화 임베딩만으로 분류 성능이 어느 정도 나오는가?

---

### 문제 3. 사전 학습된 워드 임베딩 적용  
1. **사전 학습 임베딩**: GloVe 100d (`glove.6B.100d.txt`) 사용  
2. **임베딩 매트릭스 생성**:  
   - 단어 인덱스 기반으로 GloVe 벡터 매핑  
   - OOV 단어는 랜덤 초기화  
3. **Embedding 레이어**: `trainable=False`로 고정  
4. **모델**: 문제 2와 동일한 구조  
5. **학습 및 평가**:  
   - 에포크 10, 배치사이즈 32  
   - 훈련/검증 정확도 비교  

> **질문**: 랜덤 초기화 임베딩 대비 성능 차이는?

---

### 문제 4. 신경망 언어 모델(Neural Language Model)  
1. **데이터 생성**:  
   - 전처리된 뉴스 기사 시퀀스에서 윈도우 크기 5로 슬라이딩  
   - (입력 시퀀스 길이=4, 정답=5번째 단어) 쌍 생성  
2. **모델 구조**:  
   ==> 파이토치 모델로 변경해서 구현
   - `Embedding(num_words, 100, input_length=4)`  
   - `Flatten()`  
   - `Dense(64, activation='relu')`  
   - `Dense(num_words, activation='softmax')`
   
3. **학습 설정**:  
   - 손실함수: categorical crossentropy  
   - 에포크: 5, 배치: 128  
4. **결과 기록**:  
   - 훈련 손실 및 정확도  

> **질문**: 간단한 신경망 언어 모델이 “다음 단어”를 얼마나 잘 예측하는가?

---

### 문제 5. 임베딩 벡터 시각화  
1. 문제 3에서 생성한 **사전 학습 임베딩 매트릭스** 중  
   - 상위 빈도 200개 단어 벡터 추출  
2. **차원 축소**: TSNE 또는 PCA로 2차원 변환  
3. **시각화**: matplotlib 산점도로 주요 단어(예: ‘government’, ‘market’, ‘team’, ‘technology’ 등) 라벨링  

> **질문**: 서로 의미가 비슷한 단어들은 2D 공간에서 어떻게 군집되는가?

---

### 제출 형식  
- 하나의 Jupyter Notebook(.ipynb)  
- 각 문제마다 코드, 실행 결과(숫자·그래프), 간단한 해설(1–2문장) 포함  
- 데이터 다운로드(GloVe 등)는 코드 셀에 주석으로 링크만 남기고 실제 다운로드는 수동으로 수행해도 무방  




In [1]:
# 데이터 로드
import pandas as pd

original_df = pd.read_csv("bbc-text.csv")
original_df

df = original_df.copy()
df.head(5)  # 처음 5개 행을 출력


,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [2]:
df = df.drop(columns=["category"])  # "category" 열 삭제
df.head(5)  # 처음 5개 행을 출력


,text
0,tv future in the hands of viewers with home th...
1,worldcom boss left books alone former worldc...
2,tigers wary of farrell gamble leicester say ...
3,yeading face newcastle in fa cup premiership s...
4,ocean s twelve raids box office ocean s twelve...


In [3]:
# 토큰화
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")  # punkt 패키지 다운로드
df["text"] = df["text"].apply(word_tokenize)  # "text" 열에 대해 토큰화
df.head(5)  # 처음 5개 행을 출력

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wjdgn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,text
0,"[tv, future, in, the, hands, of, viewers, with..."
1,"[worldcom, boss, left, books, alone, former, w..."
2,"[tigers, wary, of, farrell, gamble, leicester,..."
3,"[yeading, face, newcastle, in, fa, cup, premie..."
4,"[ocean, s, twelve, raids, box, office, ocean, ..."


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 텍스트 데이터 준비
texts = df["text"].apply(lambda x: " ".join(x)).tolist()

# 토크나이저 생성 및 시퀀스 변환
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# pad_sequences 적용 (길이 200)
padded_sequences = pad_sequences(sequences, maxlen=200, padding='post', truncating='post')

# 단어 집합 크기
num_words = len(tokenizer.word_index) + 1

print("단어 집합 크기:", num_words)
print("패딩 후 시퀀스 shape:", padded_sequences.shape)

단어 집합 크기: 29726
패딩 후 시퀀스 shape: (2225, 200)


In [5]:
from sklearn.model_selection import train_test_split
import numpy as np

# 훈련/검증 분할 (80/20)
X_train, X_val = train_test_split(padded_sequences, test_size=0.2, random_state=42)

print("훈련 샘플 개수:", len(X_train))
print("검증 샘플 개수:", len(X_val))

훈련 샘플 개수: 1780
검증 샘플 개수: 445


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# 모델 정의
model = Sequential([
    Embedding(input_dim=num_words, output_dim=100, input_length=200),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  # 5개 카테고리 (category_to_index 참고)
])

model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 모델 학습
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=10,
    batch_size=32
)

# 훈련/검증 정확도 및 손실 플롯
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.plot(history.history['loss'], label='Train Loss')

c:\Users\wjdgn\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


NameError: name 'y_train' is not defined

In [ ]:
# 훈련/검증 정확도 확인
train_acc = history.history['accuracy'][-1]
val_acc = history.history['val_accuracy'][-1]

print(f"랜덤 초기화 임베딩 모델의 최종 훈련 정확도: {train_acc:.4f}")
print(f"랜덤 초기화 임베딩 모델의 최종 검증 정확도: {val_acc:.4f}")

# 간단 해설
# 랜덤 초기화 임베딩만으로도 약 {val_acc:.2%}의 검증 정확도를 달성했습니다.
# 이는 임베딩이 학습 과정에서 의미 있는 표현을 일부 학습할 수 있음을 보여줍니다.

랜덤 초기화 임베딩 모델의 최종 훈련 정확도: 0.9326
랜덤 초기화 임베딩 모델의 최종 검증 정확도: 0.1775
